# TO DO:
- connect wedwards code for separating voyages
- Mess with classifier parameters (once working)
- test classifier for 2 validation sets


In [1]:
import collections

import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc

from fuzzywuzzy import fuzz
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

from exploringShipLogbooks.basic_utils import clean_data
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
from exploringShipLogbooks.basic_utils import isolate_training_data

from exploringShipLogbooks.config import *

# Load and clean data
### Load CLIWOC ship logs

In [2]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')
#cliwoc_data = cliwoc_data.loc[50000:100000]

/Applications/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Find definite slave data in CLIWOC data set
- These logs will be used to test the classifier

In [3]:
# extract logs that mention slaves
slave_mask = wc.count_key_words(cliwoc_data, text_columns, slave_words)

print('Found ', len(slave_mask[slave_mask==True]), ' logs that mention slaves')

Found  464  logs that mention slaves


### Clean CLIWOC data

In [4]:
# find indices of ship names that are "non-slave" ships before dropping ship name column
criteria = {'ShipName': non_slave_ships}
no_slave_mask = isolate_training_data(cliwoc_data, criteria)

In [5]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

### Load Slave Voyages data

In [6]:
file_name = './exploringShipLogbooks/data/tastdb-exp-2010'
slave_voyage_logs = df = pd.read_pickle(file_name)

In [7]:
year_ind = ~(slave_voyage_logs['yeardep'].isnull())
slave_voyage_logs = slave_voyage_logs[year_ind]

In [8]:
cliwoc_ind = (slave_voyage_logs['yeardep']>cliwoc_data['Year'].min()) & (slave_voyage_logs['yeardep']<cliwoc_data['Year'].max())
slave_voyage_logs = slave_voyage_logs[cliwoc_ind]

### Clean Slave voyages data

In [9]:
slave_voyage_desired_cols = ['portdep', 'portret', 'rig', 'national', 'yeardep']
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.columns = ['Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

# Join data sets

- Adding three indices to keep track of what dataset is which. We need to use this to index the classifier data later

In [10]:
cliwoc_data_indices_no_slaves = pd.DataFrame(0, index=cliwoc_data[~slave_mask].index, columns = ['data_indices'])
cliwoc_data_indices_slaves = pd.DataFrame(1, index=cliwoc_data[slave_mask].index, columns = ['data_indices'])

cliwoc_data_indices = pd.concat([cliwoc_data_indices_no_slaves, cliwoc_data_indices_slaves]).sort_index()

In [11]:
#slave_data_indices = pd.DataFrame(2, index=(slave_voyage_logs.index + cliwoc_data.tail(1).index[0]), columns = ['data_indices'])

In [12]:
slave_data_indices = pd.DataFrame(2, index=(range(len(slave_voyage_logs)) + (cliwoc_data.tail(1).index[0]+1)), columns = ['data_indices'])

- cliwoc_data (no slaves) = 0
- cliwoc_data (slaves)    = 1
- slave_data              = 2

In [13]:
indices = pd.concat([cliwoc_data_indices, slave_data_indices])

- okay, now back to your code. commented out the last lines since the above cells take care of indexing the data

In [14]:
all_data = pd.concat([cliwoc_data, slave_voyage_logs], ignore_index = True)
all_data = clean_data(all_data)

# cleanup (my computer won't run the code without this :( )
del cliwoc_data, slave_voyage_logs

## further cleaning of data

- Nationality: 
    - british and great britain
    - french and france
    - spanish and spain
    - usa and american
    
- ShipName:
    - 9740 unique shipnames good column to filter!
    
- ShipType:
    - 118743 nan values for ship types (remove these columns?)
    - duplicate types in different languages, will this be a problem?
    
- VoyageFrom:
    - 1129 unique voyage starting points
    
- VoyageTo:
    - 1074 unique voyage ending points
    
- Year:
    - filter out the ships from the slave dataset that are before 1600?

## Test of fuzzywuzzy method

In [15]:
 df = pd.DataFrame( {'id':[1, 2, 3, 4, 5, 6], 'name':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken']})

In [16]:
def func(name):
    matches = df.apply(lambda row: (fuzz.partial_ratio(row['name'], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [17]:
df.apply(lambda row: func(row['name']), axis=1)

0    [0, 3, 4]
1       [1, 2]
2       [1, 2]
3       [0, 3]
4       [0, 4]
5          [5]
dtype: object

## Try fuzzywuzzy on subset of one of our columns

In [18]:
def func(name, column_name):
    matches = all_data[0:10].apply(lambda row: (fuzz.partial_ratio(row[column_name], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [19]:
column_name = 'Nationality'
all_data[0:10].apply(lambda row: func(row[column_name], column_name), axis=1)

0    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
3    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
5    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
6    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
7    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
8    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
dtype: object

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [20]:
all_data = encode_data_df(all_data)

## Test case of combing/removing columns

In [21]:
df = pd.DataFrame( {'id2':[1, 2, 3, 4, 5, 6], 'id1':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken'], 'id3':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken'], 'id':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken']})

In [22]:
df.columns= ['id', 'id', 'id2', 'id3']

In [23]:
df.head()

,id,id,id2,id3
0,dog,dog,1,dog
1,cat,cat,2,cat
2,mad cat,mad cat,3,mad cat
3,good dog,good dog,4,good dog
4,bad dog,bad dog,5,bad dog


In [24]:
df = df.drop('id', axis=1)

In [25]:
df

,id2,id3
0,1,dog
1,2,cat
2,3,mad cat
3,4,good dog
4,5,bad dog
5,6,chicken


## Remove nan columns from encoding (currently there are no empty cells)

- if a cell were empty, all_data.loc[:,all_data.columns.get_loc('')]

In [26]:
all_data['no_data'] = all_data['nan'].apply(lambda x: x.any(), axis=1).astype(int)

In [27]:
all_data = all_data.drop('nan', axis=1)

In [28]:
all_data.head()

,american,argentina,brazil,british,danish,denmark,dutch,france,french,great britain,...,york in virginia,ysla de santa cathalina,ysla de santa cathalinaysla de santa cathalina,zeeland,ziam,zierikzee,zuid afrika,zuiden,Year,no_data
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1800,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1790,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1790,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1790,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1790,0


# Extract training data, and create list of classes

In [29]:
unclassified_logs = all_data.iloc[cliwoc_data_indices_no_slaves.index]

validation_set_1 = all_data.iloc[cliwoc_data_indices_slaves.index]

# reserve first 20% of slave_voyage_logs as validation set
validation_set_2_indices = range(min(slave_data_indices.index),
                                 min(slave_data_indices.index) + round(len(slave_data_indices)*.2))
validation_set_2 = all_data.iloc[validation_set_2_indices]

training_logs_pos = all_data.iloc[slave_data_indices.index]
training_logs_pos = training_logs_pos.drop(validation_set_2_indices)

# note! This relies on cliwoc data being first in all_data
# could make more robust later
# TODO: check that this works!! should have some in first 1000?
training_logs_neg = all_data.iloc[no_slave_mask[no_slave_mask == True].index]

# cleanup
del all_data

- left this code so we can check if there are any null values in each 
  dataframe

In [30]:
def finding_null_values(df):
    return df.isnull().sum()[df.isnull().sum()>0]

In [31]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(training_logs_neg))
classes = np.append(classes, np.ones(len(training_logs_pos)))

# joint training data
training_data = pd.concat([training_logs_neg, training_logs_pos], ignore_index = True)

# convert to numpy array
training_data = training_data.as_matrix()

# Fit training data to classifier
- **note!** first column of numpy array is index! do not include in classification!

In [32]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data[::,1::], classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Test classifier
- check if slave logs from cliwoc data were classified correctly (want mostly classified as 1)
- compare first column with slave_index


In [36]:
print('Testing validation data from slave logs data set')
validation_test(classifier, validation_set_2, 1)

print('Testing validation data from cliwoc data set:')
validation_test(classifier, validation_set_1, 1)

Testing validation data from slave logs data set
Validation set was classified as a slave ship  99.92 % of the time
Testing validation data from cliwoc data set:
Validation set was classified as a slave ship  0.0 % of the time


In [34]:
def validation_test(classifier, validation_set, expected_class):
    """
    input classifer object, validation set (data frame), and expected class 
    of validation set (i.e. 1 or 0). Prints successful classification rate.
    """
    validation_set = validation_set.as_matrix()
    predictions = classifier.predict(validation_set[::,1::])
    
    counts = collections.Counter(predictions)
    percent_correct = (counts[expected_class]/(len(predictions))* 100)
                       
    print('Validation set was classified as a slave ship ',
          round(percent_correct,2), '% of the time')